In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
from data_utils import load_cifar10 
from Trainer import *

def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):

    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]
    print(X_train.shape)

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel
    #print(mean_pixel.shape)
    return X_train, y_train, X_val, y_val, X_test, y_test




X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

(49000, 32, 32, 3)
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [2]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))

In [3]:

#import tensorflow as tf

def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):

    with tf.device(device):

        # Compute the loss like we did in Part II
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [16]:
class CustomConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2,channel_3, num_classes):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        #initializer = tf.initializers.VarianceScaling(scale=2.0)
        initializer = tf.keras.initializers.GlorotNormal()
        self.conv1 = tf.keras.layers.Conv2D(filters = channel_1, kernel_size = [3, 3],
                                           strides = 1, padding='same', kernel_initializer=initializer)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.relu1 = tf.keras.layers.ReLU()
        self.drop1 = tf.keras.layers.Dropout(rate = 0.5)
        self.max_pool1 = tf.keras.layers.MaxPool2D(pool_size = [2, 2], strides = [2, 2])
        self.conv2 = tf.keras.layers.Conv2D(filters = channel_2, kernel_size = [3, 3],
                                           strides = 1, padding='valid', kernel_initializer=initializer)
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.relu2 = tf.keras.layers.ReLU()
        self.drop2 = tf.keras.layers.Dropout(rate = 0.5)
        self.max_pool2 = tf.keras.layers.MaxPool2D(pool_size = [2, 2], strides = [2, 2])
        
        self.conv3 = tf.keras.layers.Conv2D(filters = channel_3, kernel_size = [3, 3],
                                           strides = 1, padding='valid', kernel_initializer=initializer)
        self.bn3 = tf.keras.layers.BatchNormalization()
        self.relu3 = tf.keras.layers.ReLU()
        self.drop3 = tf.keras.layers.Dropout(rate = 0.5)
        self.avg_pool3 = tf.keras.layers.AveragePooling2D(pool_size = [5, 5], strides = [1, 1])
        
        self.flatten = tf.keras.layers.Flatten()
        self.fc4 = tf.keras.layers.Dense(100, kernel_initializer=initializer)
        self.fc5 = tf.keras.layers.Dense(50, kernel_initializer=initializer)
        self.fc6 = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)
        

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        x = input_tensor
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.drop1(x)
        x = self.max_pool1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.drop2(x)
        x = self.max_pool2(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.drop3(x)
        x = self.avg_pool3(x)
        
        x = self.flatten(x)
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.fc6(x)
        
        #channel_1, channel_2, num_classes = 16, 8, 10

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        
        return x



In [ ]:
import time
tic = time.time()
print(tic)
train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)
    
    
#device = '/device:GPU:0'   # Change this to a CPU/GPU as you wish!
device = '/cpu:0'        # Change this to a CPU/GPU as you wish!
print_every = 700
num_epochs = 10
channel_1, channel_2, channel_3, num_classes = 32, 64, 128, 10
model = CustomConvNet(channel_1, channel_2, channel_3, num_classes)

def model_init_fn():
    return CustomConvNet(channel_1, channel_2, channel_3, num_classes)

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 
train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

toc = time.time()
print(toc - tic)

1585206161.7679274
Iteration 0, Epoch 1, Loss: 2.459230899810791, Accuracy: 7.8125, Val Loss: 2.3138442039489746, Val Accuracy: 7.800000190734863
Iteration 700, Epoch 1, Loss: 1.4269778728485107, Accuracy: 48.161109924316406, Val Loss: 2.801622152328491, Val Accuracy: 20.0
Iteration 1400, Epoch 2, Loss: 1.1612461805343628, Accuracy: 58.656494140625, Val Loss: 3.5124359130859375, Val Accuracy: 18.899999618530273
Iteration 2100, Epoch 3, Loss: 1.0613291263580322, Accuracy: 62.17596435546875, Val Loss: 3.523514747619629, Val Accuracy: 17.80000114440918
Iteration 2800, Epoch 4, Loss: 0.9850450158119202, Accuracy: 64.916748046875, Val Loss: 3.7371129989624023, Val Accuracy: 19.100000381469727
Iteration 3500, Epoch 5, Loss: 0.9392223358154297, Accuracy: 66.82994842529297, Val Loss: 3.6673688888549805, Val Accuracy: 17.80000114440918
Iteration 4200, Epoch 6, Loss: 0.9028566479682922, Accuracy: 68.02560424804688, Val Loss: 3.4935266971588135, Val Accuracy: 21.899999618530273
Iteration 4900, Ep

In [14]:
from __future__ import absolute_import, division, print_function, unicode_literals
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0
